In [ ]:
import cv2
import numpy as np
from tensorflow import keras

model = keras.models.load_model('best_model.h5',compile=False)

vidObj = cv2.VideoCapture('input_video.mp4')

frame_width = int(vidObj.get(3))
frame_height = int(vidObj.get(4))
fps = vidObj.get(cv2.CAP_PROP_FPS)

size = (frame_width, frame_height)

result = cv2.VideoWriter("output_video.avi", cv2.VideoWriter_fourcc(*'MJPG'),
                         fps, size)

while vidObj.isOpened():
    
    success, image = vidObj.read()
    
    if not success:
        continue
        
    h,w,_=image.shape
    img = cv2.resize(image,(128,128))
    img=np.array([img])

    pred=model.predict(img,verbose=0)
    pred=pred[0][...,0]
    pred=cv2.resize(pred,(w,h))
    pred[pred>=0.5]=255
    pred[pred<0.5]=0

    image[:,:,1]=image[:,:,1]+pred*.2
    
    result.write(image)

    cv2.imshow('video',image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
            break
vidObj.release()
result.release()
cv2.destroyAllWindows()